In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# !pip install -q langchain-openai langchain-core requests

In [4]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [72]:
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    # temperature=0.3,  # Slight increase for variation (optional)
    # max_new_tokens=100,  # Ample for complete answers
    # do_sample=True,
    # top_p=0.95,
    # repetition_penalty=1.15,
    # return_full_text=True,  # Key fix: Returns only generation, no truncation
)
hugging_face_model = ChatHuggingFace(llm=llm)

In [5]:
# tool create
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [6]:
@tool
def addition(num_1: int, num_2: int) -> int:
    """Number 1 and Number 2 are two numbers which are used to add together and thier adding is returned"""
    return num_1 + num_2

In [7]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [8]:
print(addition.invoke({'num_1' : 5, 'num_2' : 7}))

12


In [9]:
multiply.name, multiply.description, multiply.args

('multiply',
 'Given 2 numbers a and b this tool returns their product',
 {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}})

In [10]:
addition.name, addition.description, addition.args

('addition',
 'Number 1 and Number 2 are two numbers which are used to add together and thier adding is returned',
 {'num_1': {'title': 'Num 1', 'type': 'integer'},
  'num_2': {'title': 'Num 2', 'type': 'integer'}})

In [67]:
# llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")
# llm_with_tools = llm.bind_tools([addition, multiply])

llm_with_tools = hugging_face_model.bind_tools([addition, multiply])

In [12]:
query = HumanMessage('can you do addition of 9 and 9 ?')
messages = [query]
messages

[HumanMessage(content='can you do addition of 9 and 9 ?', additional_kwargs={}, response_metadata={})]

In [13]:
result = llm_with_tools.invoke(messages)
print(result)

content='' additional_kwargs={'tool_calls': [{'function': {'arguments': '{"num_1":9,"num_2":9}', 'name': 'addition', 'description': None}, 'id': 'call_63mgakqmdgzbgkrpr72f74ze', 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 346, 'total_tokens': 376}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b6862-f6c0-7ef3-8ec0-bd33413f5efa-0' tool_calls=[{'name': 'addition', 'args': {'num_1': 9, 'num_2': 9}, 'id': 'call_63mgakqmdgzbgkrpr72f74ze', 'type': 'tool_call'}] usage_metadata={'input_tokens': 346, 'output_tokens': 30, 'total_tokens': 376}


In [14]:
messages.append(result)
messages

[HumanMessage(content='can you do addition of 9 and 9 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"num_1":9,"num_2":9}', 'name': 'addition', 'description': None}, 'id': 'call_63mgakqmdgzbgkrpr72f74ze', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 346, 'total_tokens': 376}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b6862-f6c0-7ef3-8ec0-bd33413f5efa-0', tool_calls=[{'name': 'addition', 'args': {'num_1': 9, 'num_2': 9}, 'id': 'call_63mgakqmdgzbgkrpr72f74ze', 'type': 'tool_call'}], usage_metadata={'input_tokens': 346, 'output_tokens': 30, 'total_tokens': 376})]

In [15]:
tool_result = addition.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='18', name='addition', tool_call_id='call_63mgakqmdgzbgkrpr72f74ze')

In [16]:
messages.append(tool_result)
messages

[HumanMessage(content='can you do addition of 9 and 9 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"num_1":9,"num_2":9}', 'name': 'addition', 'description': None}, 'id': 'call_63mgakqmdgzbgkrpr72f74ze', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 346, 'total_tokens': 376}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b6862-f6c0-7ef3-8ec0-bd33413f5efa-0', tool_calls=[{'name': 'addition', 'args': {'num_1': 9, 'num_2': 9}, 'id': 'call_63mgakqmdgzbgkrpr72f74ze', 'type': 'tool_call'}], usage_metadata={'input_tokens': 346, 'output_tokens': 30, 'total_tokens': 376}),
 ToolMessage(content='18', name='addition', tool_call_id='call_63mgakqmdgzbgkrpr72f74ze')]

In [17]:
# final_result = llm_with_tools.invoke(messages).content
# print(final_result)
result = llm_with_tools.invoke(messages)
result

AIMessage(content='Sure, the sum of 9 and 9 is 18.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 91, 'total_tokens': 107}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6863-3ef9-73f3-ae21-296c3a2bba85-0', usage_metadata={'input_tokens': 91, 'output_tokens': 16, 'total_tokens': 107})

In [18]:
result.content

'Sure, the sum of 9 and 9 is 18.'

In [46]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [47]:
get_conversion_factor.args, convert.args

({'base_currency': {'title': 'Base Currency', 'type': 'string'},
  'target_currency': {'title': 'Target Currency', 'type': 'string'}},
 {'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
  'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}})

In [48]:
converion_factor_result = get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})
converion_factor_result

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1766966401,
 'time_last_update_utc': 'Mon, 29 Dec 2025 00:00:01 +0000',
 'time_next_update_unix': 1767052801,
 'time_next_update_utc': 'Tue, 30 Dec 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.0348}

In [49]:
base_currency_value = 10
conversion_result = convert.invoke({'base_currency_value':base_currency_value, 'conversion_rate':converion_factor_result['conversion_rate']})
print(f"{base_currency_value} Dollars Equals to {conversion_result} INR")

10 Dollars Equals to 900.3480000000001 INR


In [68]:
# tool binding
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     max_retries=6,
# )
# llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

llm_with_tools = hugging_face_model.bind_tools([get_conversion_factor, convert])

In [51]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR ?')]

In [52]:
try:
    ai_message = llm_with_tools.invoke(messages)
    messages.append(ai_message)
except Exception as e:
    print(e)

In [53]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_ddo5krdv6kbuepoclppv08yk',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_xmt6rtnxxw4j9dnzib4s0jvz',
  'type': 'tool_call'}]

In [54]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)


In [55]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor', 'description': None}, 'id': 'call_ddo5krdv6kbuepoclppv08yk', 'type': 'function'}, {'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert', 'description': None}, 'id': 'call_xmt6rtnxxw4j9dnzib4s0jvz', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 345, 'total_tokens': 397}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b6875-2059-7323-be6a-610ccaf0f6da-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_ddo5krdv6kbuepoclppv08yk

In [56]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR as of the last update is approximately 90.0348. Therefore, 10 USD is approximately equal to 900.35 INR.'

In [57]:
try: 
    from langchain.agents import initialize_agent, AgentType
    # Step 5: Initialize the Agent ---
    agent_executor = initialize_agent(
        tools=[get_conversion_factor, convert],
        llm=llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
        verbose=True  # shows internal thinking
    )
except Exception as e:
    print(e)

cannot import name 'initialize_agent' from 'langchain.agents' (g:\GenAI - CampusX\gen_ai_campusx\lib\site-packages\langchain\agents\__init__.py)


In [58]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})

NameError: name 'agent_executor' is not defined

In [ ]:
from langchain.agents import create_agent

# Your tools (must be proper LangChain Tools, e.g., decorated with @tool or StructuredTool)
tools = [get_conversion_factor, convert]

# Create the agent - this is the direct modern replacement
agent = create_agent(
    model=hugging_face_model,  # You can also pass a string like "gpt-4o" for lazy loading
    tools=tools,
    system_prompt=(
        "You are a helpful currency conversion assistant. "
        "Use the provided tools to get conversion rates and perform conversions accurately. "
        "Think step-by-step, explain your reasoning, and always provide the final answer clearly."
    ),
)

# Example usage with invoke (synchronous)
response = agent.invoke({
    "messages": [
        {"role": "user", "content": "Convert 10 USD to EUR"}
    ]
})

# Extract the final assistant response
final_message = response["messages"][-1]
print(final_message.content)

print("="*80)

# For verbose/step-by-step output (like old verbose=True), use stream:
print("\n--- Streaming output (verbose) ---")
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Convert 10 USD to EUR"}]},
    stream_mode="updates"  # Shows agent actions, tool calls, etc.
):
    print(chunk)

The current conversion factor from USD to EUR is 0.8493. This means that 1 USD is equivalent to 0.8493 EUR. 

To find out how much 10 USD is in EUR, we multiply 10 by the conversion rate:

\[ 10 \, \text{USD} \times 0.8493 \, \frac{\text{EUR}}{\text{USD}} = 8.493 \, \text{EUR} \]

Therefore, 10 USD is approximately 8.493 EUR.

--- Streaming output (verbose) ---
{'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"base_currency":"USD","target_currency":"EUR"}', 'name': 'get_conversion_factor', 'description': None}, 'id': 'call_aeuouw7kgpap0jgaljqp6nwm', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 349, 'total_tokens': 378}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b6885-a8a6-77a2-98e7-af1b5643cd42-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'targ